In [4]:
import numpy as np
import ROOT

def GetAllHistoNames(infile = ''):
    rootfile1 = infile
    f0 = ROOT.TFile(rootfile1,"READ")
    
    listofnames = []
    listoftitles = []
    
    for obj in f0.GetListOfKeys():
        listofnames.append(obj.GetName())
        listoftitles.append(obj.GetTitle())
    
    return listofnames, listoftitles


def GetLegendPlacement(position = ''):
        if position == 'r':
            placement = [0.62,0.89]
        if position == 'c':
            placement = [0.4,0.67]
        if position == 'l':
            placement = [0.19,0.47]
        return placement